In [37]:
import os
import zipfile
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
dataset_dir = "D:\\CVPI\\Dataset"
training_dir = os.path.join(dataset_dir, "training")
validation_dir = os.path.join(dataset_dir, "validation")

if not os.path.exists(training_dir) or not os.path.exists(validation_dir):
    with zipfile.ZipFile(dataset_zip, "r") as z:
        z.extractall(dataset_dir)

In [ ]:
image_size = (224, 224)
batch_size = 32

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

train_dataset = datasets.ImageFolder(root=training_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=validation_dir, transform=data_transforms['val'])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, len(class_names)) 
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [43]:
epochs = 10
save_dir = "D:\\CVPI\\SavedModels"

os.makedirs(save_dir, exist_ok=True)

start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        running_corrects += (preds == labels).sum().item()
        total_samples += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = running_corrects / total_samples
    elapsed_time = time.time() - start_time
    remaining_time = (elapsed_time / (epoch + 1)) * (epochs - (epoch + 1))

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Remaining time: {remaining_time / 60:.2f} minutes")

    torch.save(model.state_dict(), os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth'))


Epoch 1/10, Loss: 0.2094, Accuracy: 0.9298, Remaining time: 89.01 minutes
Epoch 2/10, Loss: 0.1709, Accuracy: 0.9438, Remaining time: 79.51 minutes
Epoch 3/10, Loss: 0.1477, Accuracy: 0.9505, Remaining time: 70.26 minutes
Epoch 4/10, Loss: 0.1219, Accuracy: 0.9591, Remaining time: 60.51 minutes
Epoch 5/10, Loss: 0.1073, Accuracy: 0.9645, Remaining time: 50.60 minutes
Epoch 6/10, Loss: 0.0963, Accuracy: 0.9683, Remaining time: 40.57 minutes
Epoch 7/10, Loss: 0.0837, Accuracy: 0.9715, Remaining time: 30.38 minutes
Epoch 8/10, Loss: 0.0802, Accuracy: 0.9736, Remaining time: 20.30 minutes
Epoch 9/10, Loss: 0.0697, Accuracy: 0.9767, Remaining time: 10.11 minutes
Epoch 10/10, Loss: 0.0663, Accuracy: 0.9785, Remaining time: 0.00 minutes


In [46]:
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    balanced_accuracy_score, roc_auc_score, precision_score, recall_score,
    f1_score, average_precision_score
)
from sklearn.preprocessing import label_binarize
import torch

save_dir = "D:\\CVPI\\SavedModels"
model_paths = [os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth') for epoch in range(epochs)]
class_names = ["Angioectasia", "Bleeding", "Erosion", "Erythema", "Foreign Body", 
               "Lymphangiectasia", "Normal", "Polyp", "Ulcer", "Worms"]

for epoch, model_path in enumerate(model_paths, start=1):
    model.load_state_dict(torch.load(model_path))
    model.eval()

    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            val_predictions.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_predictions)
    balanced_acc = balanced_accuracy_score(val_labels, val_predictions)
    cm = confusion_matrix(val_labels, val_predictions)
    report = classification_report(val_labels, val_predictions, target_names=class_names)
    f1 = f1_score(val_labels, val_predictions, average='macro')
    avg_precision = average_precision_score(label_binarize(val_labels, classes=range(len(class_names))), 
                                            label_binarize(val_predictions, classes=range(len(class_names))), average='macro')

    auc_roc = roc_auc_score(
        label_binarize(val_labels, classes=range(len(class_names))), 
        label_binarize(val_predictions, classes=range(len(class_names))), 
        average='macro', multi_class='ovr'
    )

    specificities = []
    for i in range(len(class_names)):
        tn = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])
        fp = cm[:, i].sum() - cm[i, i]
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificities.append(specificity)

    mean_specificity = sum(specificities) / len(specificities)

    print(f"\nModel Epoch {epoch}:")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"AUC-ROC (Macro-Average): {auc_roc:.4f}")
    print(f"Specificity (Per Class): {specificities}")
    print(f"Mean Specificity: {mean_specificity:.4f}")
    print(f"F1 Score (Macro-Average): {f1:.4f}")
    print(f"Average Precision (Macro-Average): {avg_precision:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)



Model Epoch 1:
Validation Accuracy: 0.9212
Balanced Accuracy: 0.7504
AUC-ROC (Macro-Average): 0.8663
Specificity (Per Class): [0.9969299648225136, 0.9979712166360236, 0.9740268411564399, 0.9960846226712977, 0.9972771023302939, 0.9939831528279182, 0.8738621586475943, 0.9946903787103377, 0.9979174555092768, 0.999937749003984]
Mean Specificity: 0.9823
F1 Score (Macro-Average): 0.7821
Average Precision (Macro-Average): 0.6449
Confusion Matrix:
[[  363     2    70     4     2     2    50     4     0     0]
 [    8   287    35     2     0     4    18     5     0     0]
 [   19    14   840    27    22    14   182    22    15     0]
 [    3     2    97   124     1     2    41    27     0     0]
 [    2     0    20     2   287     3    20     5     1     0]
 [    1     0    10     0     0   304    26     2     0     0]
 [    9     8    86     4    12    48 12101    18     1     0]
 [    3     6    67    23     6    20   139   236     0     0]
 [    2     0     4     0     0     2     5     0  